# Learning the basics of optimization 
We are just talking about optimization here, not spatial optimization yet. 
* YouTube video: [SciPy Beginner's guide for optimization](https://www.youtube.com/watch?v=cXHvC_FGx24&ab_channel=APMonitor.com)

In [1]:
import numpy as np
from scipy.optimize import minimize 

In [9]:
# set objective function
def objective(x): 
    # x is a list of values.
    # in our case, the list will represent all potential hub locations. The value will be either 0 (not chosen) or 1 (chosen)
    x1 = x[0]
    x2 = x[1] 
    x3 = x[2]
    x4 = x[3]
    return x1*x4*(x1+x2+x3)+x3

In [3]:
def constraint1(x): 
    return x[0]*x[1]*x[2]*x[3]-25.0
def constraint2(x): 
    sum_sq = 40 
    for i in range(4): 
        sum_sq = sum_sq - x[i]**2 
    return sum_sq

In [4]:
x0 = [1,5,5,1]
objective(x0)

16

In [5]:
b = (1,5)
bnds = (b,b,b,b)
con1 = {'type': 'ineq', 'fun': constraint1}
con2 = {'type': 'eq', 'fun': constraint2}
cons = [con1, con2]

In [6]:
sol = minimize(objective, x0, method='SLSQP', bounds=bnds, constraints=cons)

In [7]:
sol

     fun: 17.01401724563517
     jac: array([14.57227015,  1.37940764,  2.37940764,  9.56415057])
 message: 'Optimization terminated successfully'
    nfev: 25
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([1.        , 4.7429961 , 3.82115462, 1.37940765])

## Adapting tutorial code to circular hub optimization
Let's try using a very simple example - optimizing the location of just **1** hub, choosing between only **3** candidate locations. 

In [1]:
import pandas as pd
import momepy
import geopandas as gpd
import osmnx as ox
import matplotlib.pyplot as plt
import networkx as nx
from pyrosm import OSM, get_data
from shapely.geometry import Point, Polygon
from shapely.ops import nearest_points
import numpy as np
import shapely
import plotly.express as px
import time
import libpysal
from libpysal.weights import Queen, Rook, KNN, DistanceBand
import esda
import math
pd.options.mode.chained_assignment = None  # default='warn'

### reading data

In [6]:
# (simplified) road network 
df = pd.read_csv('../data/ams_roads_filtered.osm-simplified.csv', sep=';')
df = gpd.GeoDataFrame(df)
df['geometry'] = gpd.GeoSeries.from_wkt(df.wkt)
df.drop(columns='wkt', inplace=True)
df = df.set_crs('EPSG:4326')
df = df.to_crs('EPSG:28992')

# road gdf to network graph 
streets = df.copy()
graph = momepy.gdf_to_nx(streets, approach='primal')
for component in list(nx.connected_components(graph)):
    if len(component)<3:
        for node in component:
            graph.remove_node(node)

# graph to gdf of nodes and edges 
nodes, edges, sw = momepy.nx_to_gdf(graph, points=True, lines=True,
                                    spatial_weights=True)
nodesAll = nodes.geometry.unary_union

In [8]:
# land price
cbsFile = gpd.read_file('../../_bigData/cbs_popData/CBS_pc4_2020_v1.shp')
cbs = cbsFile.copy()
cbs = cbs[['WOZWONING', 'geometry']]
cbs.rename(columns={'WOZWONING': 'housePrice'}, inplace=True)
cbs.housePrice = cbs.housePrice * 1000

In [3]:
# gdf of supply of secondary resources 
supply = gpd.read_file('../../_bigData/pblUrbanMiningModels/shpsCleaned/supply_NL.shp')
xmin, ymin, xmax, ymax = edges.total_bounds
supply = supply.cx[xmin:xmax, ymin:ymax]
supply = supply.reset_index(drop=True)
supplyMelt = pd.melt(supply, id_vars=['id', 'geometry'], 
        value_vars=['steel', 'copper', 'aluminium', 'wood', 'concrete', 
                    'brick', 'glass', 'ceramic', 'plastic', 'insulat'])
supplyMelt.rename(columns={'variable': 'material', 'value': 'kg'}, inplace=True)

# gdf of demand for secondary resources
demand = gpd.read_file('../../_bigData/pblUrbanMiningModels/shpsCleaned/demand_NL.shp')
xmin, ymin, xmax, ymax = edges.total_bounds
demand = demand.cx[xmin:xmax, ymin:ymax]
demand = demand.reset_index(drop=True)
demandMelt = pd.melt(demand, id_vars=['id', 'geometry'], 
        value_vars=['aluminium', 'brick', 'concrete', 'bitumen', 'electronic', 
                    'gypsum', 'glass', 'wood', 'insulat', 'lime', 'ceramic', 'copper',
                    'plastic', 'glue', 'mortar', 'paper', 'steel', 'stone', 'sand'])
demandMelt.rename(columns={'variable': 'material', 'value': 'kg'}, inplace=True)

NameError: name 'edges' is not defined

In [ ]:
def makeGrid(cell_size, edges, buffer, plot=False): 
    # cell_size = 500
    # create square grid over NL according to cell size
    xmin, ymin, xmax, ymax = edges.total_bounds
    crs = "EPSG:28992"
    grid_cells = []
    for x0 in np.arange(xmin, xmax+cell_size, cell_size):
        for y0 in np.arange(ymin, ymax+cell_size, cell_size):
            # bounds
            x1 = x0-cell_size
            y1 = y0+cell_size
            grid_cells.append(shapely.geometry.box(x0, y0, x1, y1))
    cell = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs=crs)

    # cut square grid into shape 
    buffer = buffer
    boundary = gpd.GeoDataFrame(edges.geometry.buffer(buffer), geometry=0, crs=crs).rename(columns={0: 'geometry'})
    mergeIndex = gpd.sjoin(boundary, cell, how='left', op='intersects').index_right.unique()
    grid = cell.loc[mergeIndex]
    grid = grid.reset_index(drop=True).reset_index()
    
    # make spatial weights matrix 
    points = list(grid.apply(lambda row: (row.geometry.centroid.x, row.geometry.centroid.y), axis=1))
    threshold = math.ceil(math.sqrt((cell_size**2)*2))
    wd = libpysal.weights.DistanceBand(points ,threshold=threshold, binary=False)
    wd.transform = 'r'
    
    if plot: 
        fig, ax = plt.subplots(1,1,figsize=(10,10))
        # nodes.plot(ax=ax, color='grey')
        edges.plot(ax=ax, color='lightgrey')
        grid.boundary.plot(ax=ax, color='grey')
        ax.axis('off')
        ax.set_title('make grid according to road network')
        plt.show()
    
    return grid, wd

def get_buffer(in_data, buff=50):
    b1 = in_data.buffer(buff)  # Buffer
    ub = b1.unary_union  # Buffer Union
    b2 = gpd.GeoSeries(ub)
    out_data = gpd.GeoDataFrame(b2, crs=in_data.crs, columns=["geometry"])
    return out_data

In [ ]:
buff = 1000
streets_buffer = get_buffer(streets, buff=buff)
grid, wd = makeGrid(2000, streets, buff)
grid.geometry = grid.geometry.centroid

In [ ]:
clients, wd = makeGrid(2000, streets, 1000)
clients['totKgSupply'] = clients.geometry.map(lambda x: supply[supply.geometry.within(x)].totKg.sum())
clients['totKgDemand'] = clients.geometry.map(lambda x: demand[demand.geometry.within(x)].totKg.sum())
clients['housePrice'] = clients.geometry.map(lambda x: cbs[cbs.geometry.intersects(x)].housePrice.mean())
clients.geometry = clients.geometry.centroid

In [ ]:
clients.head()

In [ ]:
facilities = grid.sample(3, random_state=1).reset_index(drop=True)
facilities['chosen'] = 0
facilities = facilities[['chosen', 'geometry']]
facilities

In [ ]:
ax = clients.plot(markersize=clients.totKgSupply / 1000000, color='lightgrey', figsize=(8,8), alpha=0.5)
streets.plot(ax=ax, color='grey')
facilities.plot(ax=ax, color='black', marker='*', markersize=500)

In [ ]:
x0 = [1,0,0]
fac = facilities.copy() 
fac.chosen = x0
fac = fac[fac.chosen == 1]

In [ ]:
fac

In [ ]:
clients.head()

In [ ]:
def objective(x0): 
    # x = list of open / close for hubs = [1,0,0]
    
    # identify chosen facility 
    fac = facilities.copy() 
    fac.chosen = x0
    fac = fac[fac.chosen == 1]
    
    # calculate co2 reduction 
    co2Emissions = 50 # dummy number
    totSupply = clients.totKgSupply.sum()
    totDemand = clients.totKgDemand.sum()
    matKgStored = totSupply if totDemand > totSupply else totDemand
    co2Reduction = co2Emissions * matKgStored
    
    # calculate storage cost 
    landPrice = clients.loc[clients.distance(fac.geometry[0]).idxmax()].housePrice
    storageCoef = 50 # dummy 
    storageCost = landPrice * storageCoef * matKgStored
    
    # calculate transportation cost
    transPriceCoef = 50 # dummy 
    clientsDemand = clients[clients.totKgDemand > 0]
    clientsSupply = clients[clients.totKgSupply > 0]
    totDistSupply = clientsSupply.distance(fac.geometry[0]).sum() # is currently euclidean distance, need to take into account street network
    matKgSupply = clients.totKgSupply.sum() # is currently just sum of totKgSupply, without taking into account each trip 
    totDistDemand = clientsDemand.distance(fac.geometry[0]).sum()
    matKgDemand = clients.totKgDemand.sum()
    transportationCost = transPriceCoef * (totDistSupply * matKgSupply + totDistDemand * matKgDemand)
       
    # calculate transportation emissions 
    transEmissionsCoef = 50 # dummy 
    transportationEmissions = transEmissionsCoef * (totDistSupply * matKgSupply + totDistDemand * matKgDemand)
    
    # calculate cost effectiveness 
    costEffectiveness = (storageCost + transportationCost) / (co2Reduction - transportationEmissions)
    
    return costEffectiveness

In [4]:
# def constraint1(x): 
#     return x[0]*x[1]*x[2]*x[3]-25.0
# def constraint2(x): 
#     sum_sq = 40 
#     for i in range(4): 
#         sum_sq = sum_sq - x[i]**2 
#     return sum_sq

In [5]:
# b = (1,5)
# bnds = (b,b,b,b)
# con1 = {'type': 'ineq', 'fun': constraint1}
# con2 = {'type': 'eq', 'fun': constraint2}
# cons = [con1, con2]

In [ ]:
import numpy as np
from scipy.optimize import minimize 

The objective is the function that we are trying to optimize. For our case, it will be the cost effectiveness function which calculates the euro/tCO2eq for each potential scenario.

The constraints are the conditions that have to be met by the variables. For our case, this will be 

In [157]:
x0 = [1,0,0]
objective(x0)

-1.0164533941551748

In [6]:
sol = minimize(objective, x0, method='SLSQP', bounds=bnds, constraints=cons)

In [7]:
sol

     fun: 17.01401724563517
     jac: array([14.57227015,  1.37940764,  2.37940764,  9.56415057])
 message: 'Optimization terminated successfully'
    nfev: 25
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([1.        , 4.7429961 , 3.82115462, 1.37940765])